In [71]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
import warnings
import datetime

warnings.simplefilter(action = 'ignore', category = FutureWarning)
kf=KFold(n_splits=3)
v = pd.DataFrame()

import statsmodels.api as sm
from sklearn import svm

Apt_No=input('Enter Apartment number: ')
date=['Jan17','Feb17','Mar17','Apr17','May17','Jun17','Jul17','Aug17','Sep17','Oct17','Nov17','Dec17']

for d in date:
    path = 'C:\\Users\\iot\\Desktop\\DataSet\\Apt'+str(Apt_No)+'\\normalized data\\'+'Apt'+Apt_No+'_int.xlsx'
    xls_temp=pd.ExcelFile(r'C:\Users\iot\Desktop\temp_fin.xlsx')
    xls_pow=pd.ExcelFile(path)
    dataset_temp=pd.read_excel(xls_temp,d,index_col=0)
    dataset_pow=pd.read_excel(xls_pow,d,index_col=0)
    dataset_temp.drop('d_avg',inplace=True)
    dataset_temp.drop('avg',inplace=True,axis=1)
    dataset_pow.drop('d_avg',inplace=True)
    dataset_pow.drop('avg',inplace=True,axis=1)


    frames_pow=[]

    for i in range(len(dataset_pow.columns)):
        frames_pow.append(dataset_pow[i])
    keys1_pow=list(range(len(dataset_pow.columns)))
    result = pd.concat(frames_pow, keys=keys1_pow)
    result_pow = pd.DataFrame(result)

    frames_temp=[]
    for i in range(len(dataset_temp.columns)):
        frames_temp.append(dataset_temp[i])
    keys1_temp=list(range(len(dataset_temp.columns)))
    result1 = pd.concat(frames_temp, keys=keys1_temp)
    result_temp = pd.DataFrame(result1)

    final=pd.concat([result_pow,result_temp],axis=1)
    final.columns = ['Power','Temperature']
    df1 = final.reset_index(level=[0, 1])
    df1['day']=df1['level_0']+1
    df1['hours']=df1['level_1']
    df1.drop('level_0',axis=1,inplace=True)
    df1.drop('level_1',axis=1,inplace=True)

    dd5=df1['Power'].groupby([df1['day']]).mean()
    dd4=df1['Temperature'].groupby([df1['day']]).mean()
    dy=pd.concat([dd5, dd4], axis=1)
    v=v.append(dy)
    
v.index = range(365)
date1 = pd.Series(pd.date_range(start='1/1/2017', periods=365, freq='D'))
dd = pd.DataFrame(dict(date_given=date1))
dd["weekday"]=dd['date_given'].dt.dayofweek
dd["Month"]=dd['date_given'].dt.month
dd["Day"]=dd['date_given'].dt.day

df=pd.concat([dd,v],axis=1)

#a=input('How many parameters do you want to use')

#if(a=='2'):
 #   b,c=str(input('Enter parameters')).split()
  #  df_x=df[[b,c]] 
#if(a=='3'):
 #   b,c,d=str(input('Enter parameters')).split()
  #  df_x=df[[b,c,d]] 
#if(a=='4'):
 #   b,c,d,e=str(input('Enter parameters')).split()
df_x=df[['Month','Day','weekday','Temperature']] 



df_y=df['Power']

reg=linear_model.LinearRegression()

classifiers = [
    svm.SVR(),
    
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]



df_acc = pd.DataFrame({'model':[], 'acc' : []})

kf = KFold(n_splits=5)  #------------------->n_splits can be varied upto 40 to increase accuracy
                                                                           
kf.get_n_splits(df)
KFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in kf.split(df):
    x_train, x_test, y_train, y_test = train_test_split(df_x,df_y, test_size=.2,random_state=4)
    
    for item in classifiers:
    
        clf = item
        clf.fit(x_train, y_train)
        
      
        
        y_pred = clf.predict(x_test)
        err=np.mean(np.abs((y_test-y_pred)/y_test))
        accuracy = (1.0 - err)*100
        #print(item)
        #print((1.0-err)*100)
        df_acc = df_acc.append({'model':item , 'acc':accuracy} , ignore_index = True)
        #d1=pd.DataFrame({'predicted':y_pred.tolist()})
        #d2=pd.DataFrame({'original':y_test.tolist()})
        #ax = d1.plot() 
        #d1 = pd.merge(d1, d2, on=d1.index, how='outer')
              
        #d4.plot(ax=ax,title=item)
        #d1
        
        #--------------------------------------------stats model--------------------------#
        #model=sm.OLS(y_train,x_train).fit()
        #pred=model.predict(x_train)  #----------------coef  column of the table shows dependency of  Power on each variable 
        #print(model.summary())
        #--------------------------------------------
        
        
        
print('\n')
req_index = df_acc.acc.idxmax(axis = 1)
req_item = df_acc.model[req_index]
req_clf = req_item
req_clf.fit(x_train, y_train)
y_pred = req_clf.predict(x_test)
err_fin = np.mean(np.abs((y_test-y_pred)/y_test))
#print(item)
accuracy = (1.0- err_fin)*100
print('Maximum accuracy of prediction is ' + str(accuracy))
print('\n')
#d1 = pd.DataFrame({'predicted':y_pred.tolist()})
#d2 = pd.DataFrame({'original':y_test.tolist()})
#ax = d1.plot() 
#d1 = pd.merge(d1, d2, on=d1.index, how='outer')
#d2.plot(ax=ax, title = req_item)
#plt.show()

def df_day(df1, mnth_no, dy_no):
    df2 = df1[df1['month'] == mnth_no]
    df3 = df2[df2['day'] == dy_no]
    return df3

def df_month(df1, mnth_no):
    df2 = df1[df1['month'] == mnth_no]
    return df2

anlys_typ = str(input("Enter type of analyser \n (daily: 'd' , monthly: 'm' ) :  ")) 
year_no = int(input('Enter Year: '))

if (anlys_typ == 'd'):
    month_no = int(input('Enter Month : '))
    day_no = int(input('Enter Day : '))
    temp_no = float(input('Enter Expected Average Temperature: '))

if (anlys_typ == 'm'):
    month_no = int(input('Enter Month : '))
    temp_no = float(input('Enter Expected Average Temperature: '))
    

start1 = datetime.date(year_no , 1 , 1)
end1 = datetime.date(year_no, 12, 31)
date_req = pd.DataFrame(pd.date_range(start=start1, end = end1, freq='D'))
date_req.rename(columns={0:'date'},inplace=True)
date_req['month'] =date_req.date.dt.month
date_req['day'] = date_req.date.dt.day
date_req['weekday'] = date_req.date.dt.weekday
date_req['temp'] = temp_no
df_req = date_req
df_req.drop('date',inplace=True,axis=1)

temp_max = 6000
temp_min = 0
ypow_pred = req_clf.predict(df_req)
predicted_pow = (((ypow_pred - 0.3)/0.7)*(temp_max - temp_min)) + temp_min
df_req['power'] = predicted_pow*24



if (anlys_typ == 'd'):
    df_fin = df_day(df_req, month_no, day_no)

if (anlys_typ == 'm'):
    df_fin = df_month(df_req, month_no)



Enter Apartment number: 6


Maximum accuracy of prediction is 88.42975673155367


Enter type of analyser 
 (daily: 'd' , monthly: 'm' ) :  m
Enter Year: 2019
Enter Month : 5
Enter Expected Average Temperature: 46


In [72]:
tot_poww = df_fin['power'].sum()
tot_poww = tot_poww/1000

print('Expected units: ')
print(tot_poww)
print('\n')

print('Total Predicted Bill : ')
if tot_poww < 30:
    print(tot_poww*0.4)
if 30 < tot_poww < 100:
    print(tot_poww*1.9)
if 100  < tot_poww < 300:
    print(tot_poww*3.4)
if 300 < tot_poww < 1000:
    print(tot_poww*4.5)
if 1000 < tot_poww:
    print(tot_poww*7)

Expected units: 
1279.2451119799023


Total Predicted Bill : 
8954.715783859316


In [74]:
df.corr()

,weekday,Month,Day,Power,Temperature
weekday,1.000000,0.012860,-0.012152,-0.053332,-0.003486
Month,0.012860,1.000000,0.011893,-0.190052,-0.167732
Day,-0.012152,0.011893,1.000000,-0.018602,0.093002
Power,-0.053332,-0.190052,-0.018602,1.000000,0.234584
Temperature,-0.003486,-0.167732,0.093002,0.234584,1.000000
